<font color="blue">Привет. Давай смотреть как ты укротил тексты

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.utils import resample
import re
import nltk
from pathlib import Path
path = Path.cwd()

In [2]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alexeymaslov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alexeymaslov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexeymaslov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop_words = set(stopwords.words('english'))

In [4]:

def preparing():
    path = Path().absolute()
    df = pd.read_csv('{}/datasets/toxic_comments.csv'.format(path))
    return df

def train_valid_test_split(df):
    features_train, features_test, target_train, target_test = train_test_split(df['lemm_text'], df['toxic'], test_size = 0.2, random_state=12345)
    features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size = 0.25, random_state=12345)
    return features_train, features_valid, features_test, target_train, target_valid, target_test

def train_valid_test_split_df(df):
    df_train, df_test = train_test_split(df, test_size = 0.2, random_state=12345)
    df_train, df_valid = train_test_split(df_train, test_size = 0.25, random_state=12345)
    return df_train, df_valid, df_test

def features_target_split(df):
    features = df['lemm_text']
    target = df['toxic']
    return features, target

def lemmatize(m, text):
    text = clear_text(text)
    token_words=word_tokenize(text)
    lemm_text = []
    for word in token_words:
        lemm_text.append(m.lemmatize(word, pos='v'))
    return " ".join(lemm_text)

def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

def downsample(df, ratio):
    df_toxic = df[df['toxic'] == 1]
    df_not_toxic = df[df['toxic'] == 0]
    sample_size = int(df_not_toxic.shape[0] * ratio)
    df_not_toxic = df_not_toxic.sample(sample_size, random_state=12345)
    df_downsampled = pd.concat([df_toxic, df_not_toxic]) 
    return df_downsampled

def upsample(df):
    df_toxic = df[df['toxic'] == 1]
    df_not_toxic = df[df['toxic'] == 0]
    df_toxic = resample(df_toxic, replace=True, n_samples=df_not_toxic.shape[0], random_state=12345)
    df_upsampled = pd.concat([df_toxic, df_not_toxic]).sample(frac=1, random_state=12345).reset_index(drop=True)
    return df_upsampled

Лемматизируем текст

In [5]:
df = preparing()
m = WordNetLemmatizer()
df['lemm_text'] = df['text'].apply(lambda x: lemmatize(m, x))

In [6]:
df_train, df_valid, df_test = train_valid_test_split_df(df)
features_train, target_train =  features_target_split(df_train)
features_valid, target_valid = features_target_split(df_valid)
features_test, target_test = features_target_split(df_test)

In [7]:
print(df.info())
print(df.describe())
print(df.toxic.value_counts(normalize=True))

print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159571 non-null  int64 
 1   text        159571 non-null  object
 2   toxic       159571 non-null  int64 
 3   lemm_text   159571 non-null  object
dtypes: int64(2), object(2)
memory usage: 4.9+ MB
None
         Unnamed: 0          toxic
count  159571.00000  159571.000000
mean    79785.00000       0.101679
std     46064.32424       0.302226
min         0.00000       0.000000
25%     39892.50000       0.000000
50%     79785.00000       0.000000
75%    119677.50000       0.000000
max    159570.00000       1.000000
0    0.898321
1    0.101679
Name: toxic, dtype: float64
(95742,)
(31914,)
(31915,)


Всего 160К комментариев, 10% из них токсичные. Пропусков в данных нет

<font color="blue">Отлично

# 2. Обучение

Попробуем обучить линейную модель с помощью TF-IDF<br>
Будем использовать только LogisticRegression модель. Посмотрим как будет меняться F1  в зависимости от параметров векторизации и настроек модели 

In [8]:
df_train, df_valid, df_test = train_valid_test_split_df(df)
features_train, target_train =  features_target_split(df_train)
features_valid, target_valid = features_target_split(df_valid)
features_test, target_test = features_target_split(df_test)
corpus_train = features_train.values
corpus_valid = features_valid.values

Без стоп-слов; ngram из одного слова

In [9]:
count_tf_idf = TfidfVectorizer()
count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)

In [10]:
count_tf_idf.get_feature_names()[0:5]

['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaaaa']

Действительно, фичи из одного слова

In [11]:
lr_model = LogisticRegression(solver='liblinear')
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.738


In [12]:
lr_model = LogisticRegression(solver='liblinear', class_weight='balanced')
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.750


In [13]:
lr_model = LogisticRegression(solver='saga', penalty='elasticnet', l1_ratio=0.5)
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.764


In [14]:
lr_model = LogisticRegression(solver='saga', penalty='elasticnet', l1_ratio=0.3)
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.755


In [17]:
lr_model = LogisticRegression(solver='saga', penalty='l1')
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.787


Попробуем поменять порог

In [18]:
probabilities = lr_model.predict_proba(tf_idf_valid)
print(probabilities)

[[9.96979867e-01 3.02013315e-03]
 [8.35604203e-01 1.64395797e-01]
 [2.01393766e-04 9.99798606e-01]
 ...
 [9.98283139e-01 1.71686087e-03]
 [9.89344370e-01 1.06556297e-02]
 [7.42140097e-01 2.57859903e-01]]


In [19]:
probabilities_one = probabilities[:, 1]
for threshold in np.arange(0, 0.5, 0.02):
    predicted_valid = probabilities_one > threshold
    f1score = f1_score(target_valid, predicted_valid)

    print("Порог = {:.2f} | F1-score = {:.3f}".format(
        threshold, f1score))

Порог = 0.00 | F1-score = 0.188
Порог = 0.02 | F1-score = 0.393
Порог = 0.04 | F1-score = 0.542
Порог = 0.06 | F1-score = 0.641
Порог = 0.08 | F1-score = 0.690
Порог = 0.10 | F1-score = 0.723
Порог = 0.12 | F1-score = 0.742
Порог = 0.14 | F1-score = 0.759
Порог = 0.16 | F1-score = 0.768
Порог = 0.18 | F1-score = 0.779
Порог = 0.20 | F1-score = 0.783
Порог = 0.22 | F1-score = 0.786
Порог = 0.24 | F1-score = 0.789
Порог = 0.26 | F1-score = 0.794
Порог = 0.28 | F1-score = 0.794
Порог = 0.30 | F1-score = 0.795
Порог = 0.32 | F1-score = 0.794
Порог = 0.34 | F1-score = 0.793
Порог = 0.36 | F1-score = 0.794
Порог = 0.38 | F1-score = 0.794
Порог = 0.40 | F1-score = 0.794
Порог = 0.42 | F1-score = 0.791
Порог = 0.44 | F1-score = 0.791
Порог = 0.46 | F1-score = 0.789
Порог = 0.48 | F1-score = 0.787


#### Лучший результат без стоп-слов F1-score: 0.795

### Попробуем подключить словарь стоп-слов

In [20]:
print('Количество стоп-слов в словаре: {}'.format(len(stop_words)))

Количество стоп-слов в словаре: 179


In [21]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)

In [22]:
lr_model = LogisticRegression(solver='saga', penalty='l1')
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.776


In [23]:
probabilities = lr_model.predict_proba(tf_idf_valid)
probabilities_one = probabilities[:, 1]
for threshold in np.arange(0, 0.5, 0.02):
    predicted_valid = probabilities_one > threshold
    f1score = f1_score(target_valid, predicted_valid)
    print("Порог = {:.2f} | F1-score = {:.3f}".format(
        threshold, f1score))

Порог = 0.00 | F1-score = 0.188
Порог = 0.02 | F1-score = 0.359
Порог = 0.04 | F1-score = 0.506
Порог = 0.06 | F1-score = 0.633
Порог = 0.08 | F1-score = 0.688
Порог = 0.10 | F1-score = 0.719
Порог = 0.12 | F1-score = 0.740
Порог = 0.14 | F1-score = 0.754
Порог = 0.16 | F1-score = 0.767
Порог = 0.18 | F1-score = 0.775
Порог = 0.20 | F1-score = 0.779
Порог = 0.22 | F1-score = 0.784
Порог = 0.24 | F1-score = 0.789
Порог = 0.26 | F1-score = 0.791
Порог = 0.28 | F1-score = 0.789
Порог = 0.30 | F1-score = 0.789
Порог = 0.32 | F1-score = 0.791
Порог = 0.34 | F1-score = 0.789
Порог = 0.36 | F1-score = 0.788
Порог = 0.38 | F1-score = 0.788
Порог = 0.40 | F1-score = 0.787
Порог = 0.42 | F1-score = 0.785
Порог = 0.44 | F1-score = 0.782
Порог = 0.46 | F1-score = 0.780
Порог = 0.48 | F1-score = 0.779


#### Стоп-слова никак не помогли

При векторизации по дефолту все слова переводятся в нижний регистр. Предположим что в токсичных сообщениях больше слов с верхнем регистре, попробуем это учесть и не будем переводить слова в нижний регистр

In [24]:
count_tf_idf = TfidfVectorizer(lowercase=False)
count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)

In [25]:
lr_model = LogisticRegression(solver='saga', penalty='l1')
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.777


Отмена lowercase тоже не помогла<br>
#### Попробуем добавить фичи из двух слов

In [26]:
count_tf_idf = TfidfVectorizer(ngram_range=(1, 2))
count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_valid = count_tf_idf.transform(corpus_valid)

In [27]:
count_tf_idf.get_feature_names()[0:5]

['aa', 'aa aa', 'aa aat', 'aa acupuncture', 'aa again']

Действительно, появились фичи(основанные на них вектора) из двух слов

In [28]:
lr_model = LogisticRegression(solver='saga', penalty='l1')
lr_model.fit(tf_idf_train, target_train)
predictions = lr_model.predict(tf_idf_valid)
print('F1-score: {:.3f}'.format(f1_score(target_valid, predictions)))

F1-score: 0.786


In [29]:
probabilities = lr_model.predict_proba(tf_idf_valid)
probabilities_one = probabilities[:, 1]
for threshold in np.arange(0, 0.5, 0.02):
    predicted_valid = probabilities_one > threshold
    f1score = f1_score(target_valid, predicted_valid)
    print("Порог = {:.2f} | F1-score = {:.3f}".format(
        threshold, f1score))

Порог = 0.00 | F1-score = 0.188
Порог = 0.02 | F1-score = 0.367
Порог = 0.04 | F1-score = 0.504
Порог = 0.06 | F1-score = 0.611
Порог = 0.08 | F1-score = 0.668
Порог = 0.10 | F1-score = 0.704
Порог = 0.12 | F1-score = 0.729
Порог = 0.14 | F1-score = 0.750
Порог = 0.16 | F1-score = 0.761
Порог = 0.18 | F1-score = 0.769
Порог = 0.20 | F1-score = 0.776
Порог = 0.22 | F1-score = 0.781
Порог = 0.24 | F1-score = 0.785
Порог = 0.26 | F1-score = 0.788
Порог = 0.28 | F1-score = 0.789
Порог = 0.30 | F1-score = 0.791
Порог = 0.32 | F1-score = 0.792
Порог = 0.34 | F1-score = 0.794
Порог = 0.36 | F1-score = 0.795
Порог = 0.38 | F1-score = 0.793
Порог = 0.40 | F1-score = 0.793
Порог = 0.42 | F1-score = 0.791
Порог = 0.44 | F1-score = 0.790
Порог = 0.46 | F1-score = 0.789
Порог = 0.48 | F1-score = 0.787


### Лучший результат получился для для логистической регрессии solver='saga', penalty='l1', threshold=0.3. Проверим модель на тесте

In [30]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=12345)
features_train, target_train =  features_target_split(df_train)
features_test, target_test = features_target_split(df_test)
corpus_train = features_train.values
corpus_test = features_test.values

In [31]:
count_tf_idf = TfidfVectorizer()
count_tf_idf.fit(corpus_train)
tf_idf_train = count_tf_idf.transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)

In [32]:
lr_model = LogisticRegression(solver='saga', penalty='l1')
lr_model.fit(tf_idf_train, target_train)

probabilities = lr_model.predict_proba(tf_idf_test)
probabilities_one = probabilities[:, 1]
predictions = probabilities_one > 0.3

print('F1-score: {:.3f}'.format(f1_score(target_test, predictions)))

F1-score: 0.799


<font color="blue">Прям отлично и лайк лайк

# 3. Выводы

1. Удалось достичь F1-score = 0.799 даже без подключения BERT, т.е. просто по наличию конкретных слов в сообщении, без смыслового анализа
2. Подключение стоп слов не всегда помогает
3. В данном упражнении добавление би-грам никак не помогло. При этом словарь сильно увеличился(примерно в 10 раз)
4. У модели логистической регрессии куча настроек, с ними можно играться
5. Анализ текстов требует много ресурсов

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны